In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.abspath('./SplineDist/src'))

from Models.splinedist import SplineDist
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor, RichProgressBar, ModelSummary
from Datasets.DSB18 import Nuclie_datamodule 



/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:

logger = pl_loggers.TensorBoardLogger(
    save_dir="pl_logs",
    name="spline_dist",
    version=1,
    log_graph=True,
    default_hp_metric=False)

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath="Models/weights",
    filename="splineDist-{epoch:02d}-{val_loss:.2f}",
    save_top_k=5,
    mode="min",
)

earlystoping_callback = EarlyStopping("val_acc")

lr_logger = LearningRateMonitor(logging_interval="step")
summary = ModelSummary(max_depth=-1)
progressbar = RichProgressBar()

callbacks = [summary, progressbar, checkpoint_callback,
             lr_logger, earlystoping_callback]


In [7]:
model = SplineDist()
trainer = pl.Trainer(max_epochs=50,
                        gpus=0,
                        precision=16,
                        callbacks=callbacks,
                        logger=logger,
                        profiler="simple")
trainer.fit(model, Nuclie_datamodule())


Using bfloat16 Automatic Mixed Precision (AMP)
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/configuration_validator.py:118: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

    | Name                                                    | Type        | Params | In sizes                               | Out sizes         
--------------------------------------------------------------------------------------------------------------------------------------------------------
0   | segmentationBackbone                                    | UNet    